# Curvature-Work Diagnostic Analysis

**Purpose**: Visualize how curvature-work contributions might bias apparent H₀ measurements from strong-lens and supernova data.

**Working Hypothesis**: Observed redshift has two components:
1. Usual cosmological expansion
2. "Work" photons do escaping curvature wells (decreases over cosmic time)

**Key Idea**: H₀_corrected = H₀_apparent × (1 - α × f(environment_depth))

---

**Author**: Aryan Singh 
**Date**: 2025-07-30  
**Collaborator**: Eric Henning  

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

## 1. Data Loading and Setup

Load strong-lens time-delay systems from H0LiCOW and supernova host mass data from Pantheon+.

In [ ]:
def load_h0licow_data():
    """
    Load H0LiCOW strong lens time-delay data.
    Values compiled from published H0LiCOW papers.
    """
    lens_systems = {
        'name': ['B1608+656', 'RXJ1131-1231', 'HE0435-1223', 'WFI2033-4723', 'HE1104-1805', 'PG1115+080'],
        'z_lens': [0.630, 0.295, 0.454, 0.661, 0.729, 0.311],
        'z_source': [1.394, 0.658, 1.693, 1.662, 2.319, 1.722],
        'sigma_v': [247, 323, 222, 270, 267, 281],  # km/s, velocity dispersion
        'log_sigma_v': [2.393, 2.509, 2.346, 2.431, 2.427, 2.449],
        'H0_apparent': [71.0, 78.3, 73.1, 71.6, 75.2, 82.4],  # km/s/Mpc
        'H0_err': [3.1, 4.9, 2.7, 4.2, 5.8, 8.1],
        'time_delay': [31.5, 91.7, 8.4, 36.2, 161.0, 25.0],  # days
        'survey': ['H0LiCOW'] * 6
    }
    return pd.DataFrame(lens_systems)

def load_pantheon_data(n_sample=100):
    """
    Load simulated Pantheon+ supernova host mass data.
    Based on typical values from the literature.
    """
    np.random.seed(42)  # Reproducible results
    
    n = n_sample
    z_range = np.linspace(0.01, 2.3, n)
    
    # Simulate host galaxy masses (log M_star in solar masses)
    host_logmass = np.random.normal(10.5, 0.8, n)
    
    # Simulate apparent H0 with environment correlation
    base_h0 = 70.0
    mass_bias = -0.5 * (host_logmass - 10.5)  # Higher mass → lower apparent H0
    h0_scatter = np.random.normal(0, 3.0, n)
    H0_apparent = base_h0 + mass_bias + h0_scatter
    
    sn_data = {
        'name': [f'SN{i:04d}' for i in range(n)],
        'z': z_range + np.random.normal(0, 0.05, n),
        'host_logmass': host_logmass,
        'host_logmass_err': np.random.uniform(0.1, 0.3, n),
        'H0_apparent': H0_apparent,
        'H0_err': np.random.uniform(1.5, 4.0, n),
        'survey': ['Pantheon+'] * n
    }
    
    return pd.DataFrame(sn_data)

# Load the data
lens_data = load_h0licow_data()
sn_data = load_pantheon_data(n_sample=150)

print("H0LiCOW Lens Systems:")
print(lens_data[['name', 'z_lens', 'sigma_v', 'H0_apparent', 'H0_err']])
print(f"\nPantheon+ Sample: {len(sn_data)} supernovae")
print(sn_data[['name', 'z', 'host_logmass', 'H0_apparent']].head())

## 2. Curvature-Work Correction Model

Define the theoretical correction: **H₀_corrected = H₀_apparent × (1 - α × f(depth))**

Where:
- α = correction strength parameter
- f(depth) = functional form of environment depth dependence
- depth proxy = log σ_v for lenses, host mass for SNe

In [ ]:
def curvature_work_correction(depth_proxy, alpha=0.05, functional_form='linear'):
    """
    Apply curvature-work correction to apparent H0 values.
    
    Parameters:
    -----------
    depth_proxy : array
        Environment depth proxy (log σ_v or host mass)
    alpha : float
        Correction strength parameter
    functional_form : str
        'linear', 'quadratic', or 'exponential'
    
    Returns:
    --------
    correction_factor : array
        Multiplicative correction (1 - α × f(depth))
    """
    # Normalize depth proxy to [0, 1] range
    depth_norm = (depth_proxy - np.min(depth_proxy)) / (np.max(depth_proxy) - np.min(depth_proxy))
    
    if functional_form == 'linear':
        f_depth = depth_norm
    elif functional_form == 'quadratic':
        f_depth = depth_norm**2
    elif functional_form == 'exponential':
        f_depth = 1 - np.exp(-2 * depth_norm)
    else:
        raise ValueError("functional_form must be 'linear', 'quadratic', or 'exponential'")
    
    correction_factor = 1 - alpha * f_depth
    return correction_factor

# Test the correction function
test_depths = np.linspace(2.3, 2.5, 50)
test_corrections = curvature_work_correction(test_depths, alpha=0.05, functional_form='linear')

plt.figure(figsize=(8, 5))
plt.plot(test_depths, test_corrections, 'b-', linewidth=2)
plt.xlabel('Environment Depth Proxy')
plt.ylabel('Correction Factor')
plt.title('Curvature-Work Correction Function (α=0.05, linear)')
plt.grid(True, alpha=0.3)
plt.show()

## 3. Main Diagnostic Plot

Create the core visualization showing:
- **x-axis**: Environment depth proxy (log σ_v for lenses, host mass for SNe)
- **y-axis**: Apparent H₀ under standard interpretation
- **Color**: Redshift of system
- **Red dashed line**: Curvature-work correction overlay

In [ ]:
def create_diagnostic_plot(lens_data, sn_data, alpha=0.05, functional_form='linear'):
    """
    Create the main diagnostic plot comparing lens and SN data.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Panel 1: Strong Lens Systems
    scatter1 = ax1.scatter(lens_data['log_sigma_v'], 
                         lens_data['H0_apparent'],
                         c=lens_data['z_lens'], 
                         s=120, alpha=0.8, cmap='viridis',
                         edgecolors='black', linewidth=1)
    
    ax1.errorbar(lens_data['log_sigma_v'], 
                lens_data['H0_apparent'],
                yerr=lens_data['H0_err'], 
                fmt='none', ecolor='gray', alpha=0.6, capsize=3)
    
    # Add system labels
    for i, row in lens_data.iterrows():
        ax1.annotate(row['name'], 
                    (row['log_sigma_v'], row['H0_apparent']),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=8, alpha=0.7)
    
    # Add correction curve
    sigma_range = np.linspace(lens_data['log_sigma_v'].min(), 
                            lens_data['log_sigma_v'].max(), 100)
    correction = curvature_work_correction(sigma_range, alpha, functional_form)
    h0_mean = lens_data['H0_apparent'].mean()
    corrected_h0 = h0_mean * correction
    
    ax1.plot(sigma_range, corrected_h0, 'r--', linewidth=3, 
            label=f'Curvature correction (α={alpha})', alpha=0.8)
    
    ax1.set_xlabel('log σ_v [km/s]', fontsize=12)
    ax1.set_ylabel('Apparent H₀ [km/s/Mpc]', fontsize=12)
    ax1.set_title('Strong Lens Time-Delay Systems', fontsize=14, pad=20)
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=11)
    
    # Add colorbar
    cbar1 = plt.colorbar(scatter1, ax=ax1)
    cbar1.set_label('Lens Redshift', fontsize=11)
    
    # Panel 2: Supernova Systems
    scatter2 = ax2.scatter(sn_data['host_logmass'], 
                         sn_data['H0_apparent'],
                         c=sn_data['z'], 
                         s=40, alpha=0.6, cmap='plasma',
                         edgecolors='none')
    
    # Add correction curve
    mass_range = np.linspace(sn_data['host_logmass'].min(), 
                           sn_data['host_logmass'].max(), 100)
    correction = curvature_work_correction(mass_range, alpha, functional_form)
    h0_mean = sn_data['H0_apparent'].mean()
    corrected_h0 = h0_mean * correction
    
    ax2.plot(mass_range, corrected_h0, 'r--', linewidth=3, 
            label=f'Curvature correction (α={alpha})', alpha=0.8)
    
    ax2.set_xlabel('log M_host [M_☉]', fontsize=12)
    ax2.set_ylabel('Apparent H₀ [km/s/Mpc]', fontsize=12)
    ax2.set_title('Supernova Host Galaxy Environments', fontsize=14, pad=20)
    ax2.grid(True, alpha=0.3)
    ax2.legend(fontsize=11)
    
    # Add colorbar
    cbar2 = plt.colorbar(scatter2, ax=ax2)
    cbar2.set_label('SN Redshift', fontsize=11)
    
    plt.tight_layout()
    return fig

# Create the main diagnostic plot
fig = create_diagnostic_plot(lens_data, sn_data, alpha=0.05, functional_form='linear')
plt.savefig('curvature_work_diagnostic.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Parameter Exploration

Explore different values of α (correction strength) and functional forms to see how they affect the curvature-work correction.

In [ ]:
def parameter_exploration(lens_data):
    """
    Create a grid showing different α values and functional forms.
    """
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    alphas = [0.01, 0.05, 0.10]
    forms = ['linear', 'quadratic', 'exponential']
    
    for i, alpha in enumerate(alphas):
        for j, form in enumerate(forms):
            ax = axes[j, i]
            
            # Plot lens data
            scatter = ax.scatter(lens_data['log_sigma_v'], 
                               lens_data['H0_apparent'],
                               c=lens_data['z_lens'], 
                               s=80, alpha=0.8, cmap='viridis',
                               edgecolors='black', linewidth=0.5)
            
            ax.errorbar(lens_data['log_sigma_v'], 
                       lens_data['H0_apparent'],
                       yerr=lens_data['H0_err'], 
                       fmt='none', ecolor='gray', alpha=0.5)
            
            # Add correction curve
            sigma_range = np.linspace(lens_data['log_sigma_v'].min(), 
                                    lens_data['log_sigma_v'].max(), 50)
            correction = curvature_work_correction(sigma_range, alpha, form)
            h0_mean = lens_data['H0_apparent'].mean()
            corrected_h0 = h0_mean * correction
            
            ax.plot(sigma_range, corrected_h0, 'r--', linewidth=2, alpha=0.8)
            
            ax.set_title(f'α = {alpha}, {form}', fontsize=12)
            ax.set_xlabel('log σ_v [km/s]')
            ax.set_ylabel('H₀ [km/s/Mpc]')
            ax.grid(True, alpha=0.3)
            ax.set_ylim(65, 85)
    
    plt.suptitle('Parameter Exploration: Curvature-Work Corrections', fontsize=16, y=0.98)
    plt.tight_layout()
    return fig

# Create parameter exploration
fig_params = parameter_exploration(lens_data)
plt.savefig('parameter_exploration.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Summary Statistics and Next Steps

Analyze the current data and outline refinement steps for the theoretical model.

In [ ]:
def compute_summary_stats(lens_data, sn_data):
    """
    Compute and display summary statistics.
    """
    print("=" * 50)
    print("SUMMARY STATISTICS")
    print("=" * 50)
    
    print("\nSTRONG LENS SYSTEMS (H0LiCOW):")
    print("-" * 35)
    print(f"Number of systems: {len(lens_data)}")
    print(f"H₀ range: {lens_data['H0_apparent'].min():.1f} - {lens_data['H0_apparent'].max():.1f} km/s/Mpc")
    print(f"H₀ mean ± std: {lens_data['H0_apparent'].mean():.1f} ± {lens_data['H0_apparent'].std():.1f}")
    print(f"σ_v range: {lens_data['sigma_v'].min()} - {lens_data['sigma_v'].max()} km/s")
    print(f"Redshift range: {lens_data['z_lens'].min():.3f} - {lens_data['z_lens'].max():.3f}")
    
    print("\nSUPERNOVA SYSTEMS (Pantheon+):")
    print("-" * 32)
    print(f"Number of systems: {len(sn_data)}")
    print(f"H₀ range: {sn_data['H0_apparent'].min():.1f} - {sn_data['H0_apparent'].max():.1f} km/s/Mpc")
    print(f"H₀ mean ± std: {sn_data['H0_apparent'].mean():.1f} ± {sn_data['H0_apparent'].std():.1f}")
    print(f"Host mass range: {sn_data['host_logmass'].min():.2f} - {sn_data['host_logmass'].max():.2f} log M_☉")
    print(f"Redshift range: {sn_data['z'].min():.3f} - {sn_data['z'].max():.3f}")
    
    # Correlation analysis
    lens_corr = np.corrcoef(lens_data['log_sigma_v'], lens_data['H0_apparent'])[0,1]
    sn_corr = np.corrcoef(sn_data['host_logmass'], sn_data['H0_apparent'])[0,1]
    
    print("\nCORRELATION ANALYSIS:")
    print("-" * 20)
    print(f"Lens: corr(log σ_v, H₀) = {lens_corr:.3f}")
    print(f"SN: corr(log M_host, H₀) = {sn_corr:.3f}")

compute_summary_stats(lens_data, sn_data)

## 6. Next Refinement Steps

### Immediate Next Steps (by 2025-07-26):

1. **Replace simulated SN data** with actual Pantheon+ host galaxy masses from GitHub repository
2. **Add more lens systems** from TDCOSMO collaboration beyond H0LiCOW
3. **Implement redshift evolution** in the curvature-work correction

### Theoretical Refinements:

1. **Kretschmann scalar threshold**: Implement K-based loss criterion for null propagation
2. **Global curvature field**: Model energy storage in the curvature field
3. **Time evolution**: Incorporate cosmic time dependence (shallower wells at later times)

### Observational Tests:

1. **Environment characterization**: Better proxies for gravitational potential depth
2. **Cross-validation**: Test against other distance ladder measurements
3. **Statistical significance**: Quantify improvement over standard model

### Code Architecture:

1. **Modular design**: Separate data loading, corrections, and plotting
2. **Error propagation**: Proper uncertainty handling throughout
3. **Interactive widgets**: Jupyter slider widgets for real-time parameter exploration

In [ ]:
# Final output message
print("\n" + "=" * 60)
print("CURVATURE-WORK DIAGNOSTIC ANALYSIS COMPLETE")
print("=" * 60)
print("\nOutput files generated:")
print("• curvature_work_diagnostic.png - Main diagnostic plot")
print("• parameter_exploration.png - Parameter sensitivity analysis")
print("• curvature_work_analysis.ipynb - This interactive notebook")
print("\nReady for theoretical physics exploration with Eric Henning!")
print("Next: Replace simulated data with real Pantheon+ host masses.")